In [151]:
import numpy as np
import pandas as pd
from pathlib import Path

In [152]:
train_df = pd.read_csv(Path('Resources/2019loans.csv'))
test_df = pd.read_csv(Path('Resources/2020Q1loans.csv'))

In [153]:
train_df.head()

,Unnamed: 0,index,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,loan_status,pymnt_plan,...,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag
0,57107,57107,13375.0,0.1797,483.34,MORTGAGE,223000.0,Not Verified,low_risk,n,...,100.0,50.0,0.0,0.0,577150.0,122018.0,32000.0,170200.0,N,N
1,141451,141451,21000.0,0.1308,478.68,MORTGAGE,123000.0,Source Verified,low_risk,n,...,85.0,33.3,0.0,0.0,132750.0,27896.0,15900.0,35398.0,N,N
2,321143,321143,20000.0,0.1240,448.95,MORTGAGE,197000.0,Source Verified,low_risk,n,...,85.7,33.3,0.0,0.0,628160.0,114043.0,22600.0,90340.0,N,N
3,11778,11778,3000.0,0.1240,100.22,RENT,45000.0,Not Verified,low_risk,n,...,100.0,16.7,1.0,0.0,42006.0,20761.0,19900.0,15406.0,N,N
4,169382,169382,30000.0,0.1612,1056.49,MORTGAGE,133000.0,Source Verified,low_risk,n,...,100.0,66.7,0.0,0.0,283248.0,109056.0,79500.0,58778.0,N,N


In [154]:
#drop unnamed collumn
train_df = train_df.drop('Unnamed: 0', axis = 1).set_index('index')
#Prepare for trainging data conversion
x_train = train_df.drop('loan_status', axis=1)

In [182]:
train_df["loan_status"].value_counts()

high_risk    6090
low_risk     6090
Name: loan_status, dtype: int64

In [184]:
test_df.head()

,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,loan_status,pymnt_plan,dti,delinq_2yrs,...,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag
index,,,,,,,,,,,,,,,,,,,,,
67991,40000.0,0.0819,814.70,MORTGAGE,140000.0,Not Verified,low_risk,n,19.75,0.0,...,97.7,0.0,0.0,0.0,527975.0,70914.0,74600.0,99475.0,N,N
25429,6000.0,0.1524,208.70,RENT,55000.0,Not Verified,low_risk,n,11.52,2.0,...,66.7,0.0,0.0,0.0,34628.0,23460.0,5900.0,23628.0,N,N
38496,3600.0,0.1695,128.27,RENT,42000.0,Not Verified,low_risk,n,6.74,0.0,...,100.0,0.0,0.0,0.0,23100.0,19183.0,7300.0,15000.0,N,N
19667,20000.0,0.1524,478.33,RENT,100000.0,Not Verified,low_risk,n,12.13,0.0,...,100.0,50.0,0.0,0.0,56481.0,43817.0,13800.0,35981.0,N,N
37505,3600.0,0.1240,120.27,RENT,50000.0,Not Verified,low_risk,n,16.08,0.0,...,100.0,25.0,0.0,0.0,45977.0,32448.0,21000.0,24977.0,N,N


# Prediction: Linear vs Forest 
# You will be creating and comparing two models on this data: a logistic regression, and a random forests classifier. Before you create, fit, and score the models, make a prediction as to which model you think will perform better.

1) beleive while random forest classifire model is more of accuracy driven algorithm, it's efficiency depends on the best and proper fit data set. Since we got the data from generateddata.ipynb and used techniques like "Undersampling", "Oversampling" and "Smote" to fix imbalanced dataset, Linear regression should be better in precision. 
2) In a position where I am working for finencial establishment, and I'm giving out loans. Finance departments are always under regulatory scrutiny; Linear regression model should be easily explained and over all perfrom better.
    

In [157]:
# drop unnamed column
test_df = test_df.drop('Unnamed: 0', axis = 1).set_index('index')
#Prepare for testing data conversion
x_test = test_df.drop('loan_status', axis=1)

In [158]:
test_df["loan_status"].value_counts()

high_risk    2351
low_risk     2351
Name: loan_status, dtype: int64

In [159]:
# Convert categorical data to numeric and separate target feature for training data
from sklearn.preprocessing import LabelEncoder, StandardScaler
x_train_d = pd.get_dummies(x_train)
#convert to array
y_train_l = LabelEncoder().fit_transform(train_df['loan_status'])
y_train_l


array([1, 1, 1, ..., 0, 0, 0])

In [160]:
# Convert categorical data to numeric and separate target feature for testing data
x_test_d = pd.get_dummies(x_test)
#convert to array
y_test_l = LabelEncoder().fit_transform(test_df['loan_status'])
y_test_l

array([1, 1, 1, ..., 0, 0, 0])

In [169]:
# add missing dummy variables to testing set, Set default to 0
missing_d = set(x_train_d.columns) - set(x_test_d.columns)
for x in missing_d:
    x_test_d[x] = 0
x_test_d = x_test_d[x_train_d.columns]

In [170]:
x_test_d.head()

,loan_amnt,int_rate,installment,annual_inc,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,revol_bal,...,verification_status_Verified,pymnt_plan_n,initial_list_status_f,initial_list_status_w,application_type_Individual,application_type_Joint App,hardship_flag_N,hardship_flag_Y,debt_settlement_flag_N,debt_settlement_flag_Y
index,,,,,,,,,,,,,,,,,,,,,
67991,40000.0,0.0819,814.70,140000.0,19.75,0.0,1.0,18.0,0.0,9471.0,...,0,1,0,1,1,0,1,0,1,0
25429,6000.0,0.1524,208.70,55000.0,11.52,2.0,0.0,8.0,0.0,1280.0,...,0,1,0,1,1,0,1,0,1,0
38496,3600.0,0.1695,128.27,42000.0,6.74,0.0,0.0,6.0,0.0,4757.0,...,0,1,0,1,1,0,1,0,1,0
19667,20000.0,0.1524,478.33,100000.0,12.13,0.0,2.0,7.0,0.0,12731.0,...,0,1,0,1,1,0,1,0,1,0
37505,3600.0,0.1240,120.27,50000.0,16.08,0.0,3.0,6.0,0.0,10413.0,...,0,1,0,1,1,0,1,0,1,0


In [176]:
# Train the Logistic Regression model on the unscaled data and print the model score
from sklearn.linear_model import LogisticRegression
classify_lr = LogisticRegression()
# Use training data into model
classify_lr.fit(x_train_d, y_train_l)
#Print model score
print(f"Training Score: {classify_lr.score(x_train_d, y_train_l)}")
print(f"Testing Score: {classify_lr.score(x_test_d, y_test_l)}")


Training Score: 0.6506568144499179
Testing Score: 0.5168013611229264


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [177]:
# Train a Random Forest Classifier model and print the model score
from sklearn.ensemble import RandomForestClassifier
classify_rf = RandomForestClassifier(random_state=1, n_estimators=100).fit(x_train_d, y_train_l)
print(f'Training Score: {classify_rf.score(x_train_d, y_train_l)}')
print(f'Testing Score: {classify_rf.score(x_test_d, y_test_l)}')

Training Score: 1.0
Testing Score: 0.6544023819651212


In [178]:
# Scale the data
scale = StandardScaler().fit(x_train_d)
#set variables to train and test scale
x_train_scale = scale.transform(x_train_d)
x_test_scale = scale.transform(x_test_d)

Notice after Scaling the data, Linear regression model;training and testing score had both increased. More accurate and more precise.

In [179]:
# Train the Logistic Regression model on the scaled data and print the model score
classify_lr = LogisticRegression().fit(x_train_scale, y_train_l)
print(f'Training Score: {classify_lr.score(x_train_scale, y_train_l)}')
print(f'Testing Score: {classify_lr.score(x_test_scale, y_test_l)}')

Training Score: 0.7078817733990148
Testing Score: 0.767333049766057


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


After scaling the data, Random Forest Classifier model; training score remained the same, however testing score had decreased, making it let precise.

In [180]:
# Train a Random Forest Classifier model on the scaled data and print the model score
classify_rf = RandomForestClassifier(random_state=1, n_estimators=250).fit(x_train_scale, y_train_l)
print(f'Training Score: {classify_rf.score(x_train_scale, y_train_l)}')
print(f'Testing Score: {classify_rf.score(x_test_scale, y_test_l)}')

Training Score: 1.0
Testing Score: 0.6471714164185453
